In [ ]:
!pip install -qU "langchain[together]" langchain-huggingface


In [ ]:
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings


In [ ]:
DATA_PATH = "../data"

def load_documents():
    loader = TextLoader(DATA_PATH, glob="*.txt")
    documents = loader.load()
    return documents

documents = load_documents()

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=300,
    chunk_overlap=20,
    separators=["\n\n","\n",".","?","!"," ",""],
    length_function=len
)

chunks = documents.text_splitter.split_documents()


In [ ]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
